In [1]:
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from pyscf.geomopt.berny_solver import optimize
from berny import Berny, geomlib,Geometry,Math
import berny.coords
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM,FcM_like
from alch_deriv import alch_deriv
from Morse import mpa, Morse_interpolator
from AP_class import APDFT_perturbator as AP
from FDcoeffs import *
from ABSE import absec
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [2]:
%load_ext autoreload
%autoreload 2
from AP_class import APDFT_perturbator as AP


In [3]:
absec("C","N")

-0.0004869952092718677

In [4]:
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2]))*180/np.pi,decimals=3)

In [5]:
def nr_step(ic,g,h):
    ncoords=ic.shape[0]*ic.shape[1]
    p1cds=ic-np.linalg.solve((np.swapaxes(h,1,2)).reshape(ncoords,ncoords),g.reshape(-1)).reshape(ic.shape[0],ic.shape[1])
    p1cds-=p1cds[0]
    print (norm(p1cds[1]))
    if p1cds.shape[0]>2:
        print (anglecenter(p1cds),anglecenter(p1cds)*np.pi/180)
    return(p1cds)

In [6]:
def build_h_ic(s,g_ic,h0,B_inv):
    geom0=s.geom.copy()
    B=s.coords.B_matrix(geom0)
    bms=[]
    for i in range(geom0.coords.flatten().shape[0]):
        a=geom0.coords.flatten()
        geom1=geom0.copy()
        a[i]+=.001*bohr2ang
        a=a.reshape(geom0.coords.shape)
        geom1.coords=a
        bms.append((s.coords.B_matrix(geom1)-B)*1000)
    bms_arr=np.asarray(bms)
    BpG2=np.einsum('ijk,j->ik',bms,g_ic)
    h_ic=B_inv.T@(h0-BpG2)@B_inv
    return h_ic

tbbde={"CH":105/627.5,"HC":105/627.5,"NH":110/627.5,"HN":110/627.5,\
       "OH":119/627.5,"HO":119/627.5,"FH":136/627.5,"HF":136/627.5}

ebde=100/627.5 # empirical bond dissociation energy

def mpa_pb(coords,atoml,g,h,gic=False,ghost=[]):
    g=np.asarray(g)
    if not len(h.shape)==2:
        h=h.swapaxes(1,2)
        h=h.reshape(g.shape[0]*3,g.shape[0]*3)
    geom0=Geometry(atoml,coords*bohr2ang)
    bernyobj=Berny(geom0)
    s=bernyobj._state
    B = s.coords.B_matrix(geom0)
    q0=s.coords.eval_geom(geom0)
    B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
    if not gic:
        g_ic=np.dot(B_inv.T, (g).reshape(-1))
    else:
        g_ic=g
    h_ic=build_h_ic(s,g_ic,h,B_inv)

    bnr=0
    ddq_mb=np.zeros_like(q0)
    for i in range(len(s.coords._coords)):
        if s.coords._coords[i].__class__ is berny.coords.Bond:
            bnr+=1
            if s.coords._coords[i].i not in ghost and s.coords._coords[i].j not in ghost:
                ddq_mb[i]+=mpa(g_ic[i],h_ic[i,i],q0[i],ebde)[0]+g_ic[i]/h_ic[i,i]-q0[i]
    dq_NR=-np.linalg.solve(h_ic,g_ic)
    ddq_mb[bnr:]=np.linalg.solve(h_ic[bnr:,bnr:],-h_ic[bnr:,:]@(ddq_mb))
    return q0,dq_NR,q0+dq_NR,q0+dq_NR+ddq_mb

In [185]:
# include energy prediction 
def mpa_pb_with_energy(coords,atoml,g,h,e,gic=False,ghost=[]):
    g=np.asarray(g)
    if not len(h.shape)==2:
        h=h.swapaxes(1,2)
        h=h.reshape(g.shape[0]*3,g.shape[0]*3)
    geom0=Geometry(atoml,coords*bohr2ang)
    bernyobj=Berny(geom0)
    s=bernyobj._state
    B = s.coords.B_matrix(geom0)
    q0=s.coords.eval_geom(geom0)
    B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
    if not gic:
        g_ic=np.dot(B_inv.T, (g).reshape(-1))
    else:
        g_ic=g
    h_ic=build_h_ic(s,g_ic,h,B_inv)
    bnr=0
    dde_mb=0
    ddq_mb=np.zeros_like(q0)
    for i in range(len(s.coords._coords)):
        if s.coords._coords[i].__class__ is berny.coords.Bond:
            bnr+=1
            if s.coords._coords[i].i not in ghost and s.coords._coords[i].j not in ghost:
                MorseInt=Morse_interpolator(g_ic[i],h_ic[i,i],q0[i],ebde,e)
                ddq_mb[i]+=MorseInt.re+g_ic[i]/h_ic[i,i]-q0[i]
                dde_mb+=MorseInt.e_min-e+1/2*h_ic[i,i]*(MorseInt.re-q0[i])**2
    dq_NR=-np.linalg.solve(h_ic,g_ic)
    ddq_mb[bnr:]=np.linalg.solve(h_ic[bnr:,bnr:],-h_ic[bnr:,:]@(ddq_mb))
    dq_tot=dq_NR+ddq_mb
    en_MB=(e-1/2*np.einsum("i,ij,j",dq_tot,h_ic,dq_tot)+dde_mb)
    print (s.coords._coords)
    return q0,q0+dq_tot,en_MB
    return q0,dq_NR,q0+dq_NR,q0+dq_tot,en_MB

In [8]:
a=np.array([1,2,3])
b=np.array([1,2,3])
np.hstack([a,b])

array([1, 2, 3, 1, 2, 3])

La procedura è:
<ol>
<li> Calculate predicted Hessian and gradient </li>
<li> Bust the Ghost atom problem, the series converges slowly, problem to move ghost "too flexible" </li>
<li> Force to zero the gradient on the ghost </li>
<li> To maintain zero sum forces (are all internal) sum force on the ghot to the atom which is bounded </li>   
<li> reduce by one the number of atoms excluding the ghost </li>
<li> transform gradient and hessian to the IC uing pyberny </li>
<li> Take Newton ralphson step and the morse potential interpolation in the internal coordinaates  </li>
<li> Use a correction to the monodimentional contribution   </li>
<li> Update the other coordinantes to the new bond lengths  </li>
<ol/>

In [167]:
#LOad optimized geometries
ch4coords,nh3optg,oh2optg,hfoptg=np.load("optgeoms.npy")
ch4en,nh3en,h2oen,hfen=np.load("minim_en.npy")
af_3ch4,ah_2ch4,af_3,ah_2,AF_3,AH_2,ep_h20_hf,ep_nh3_h20,ep_ch4_nh3=np.load("ch4_nh3_data.npy")

In [166]:
#np.save("minim_en",[ch4en,nh3en,h2oen,hfen])

In [190]:
ch4en,mpa_pb_with_energy(ch4coords[:4],["N","H","H","H"],af_3ch4[:4],ah_2ch4[:4,:4,:,:],ep_ch4_nh3[-1]),nh3en

[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Bond(0, 3, weak=0), Angle(1, 0, 2, weak=0), Angle(1, 0, 3, weak=0), Angle(2, 0, 3, weak=0)]


(-40.2151804034983,
 (array([2.04385424, 2.04385424, 2.04385424, 1.91063324, 1.91063324,
         1.91063324]),
  array([1.8785112 , 1.87851289, 1.87851545, 1.91817704, 1.91817788,
         1.91817798]),
  -56.22820874197485),
 -56.2221836146943)

In [189]:
1.91063324*180/np.pi,1.91817788*180/np.pi #CH4 angle and prediction of NH3

(109.4712208494061, 109.90349687935168)

In [176]:
mpa_pb_with_energy(nh3optg[:3],["O","H","H"],af_3[:3],ah_2[:3,:3,:,:],ep_nh3_h20[-1]),h2oen

[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Angle(1, 0, 2, weak=0)]


((array([1.88552   , 1.88552   , 1.88833539]),
  array([1.7774034 , 1.7774046 , 1.86818723]),
  -76.07005350225101),
 -76.0633193328123)

In [187]:
1.88833539*180/np.pi,1.86818723*180/np.pi #NH3 angle and prediction of H2O

(108.19364815219032, 107.03924361923602)

In [180]:
mpa_pb_with_energy(oh2optg,["F","H","H"],AF_3,AH_2,ep_h20_hf[-1],ghost=[1,2]),hfen

[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Angle(1, 0, 2, weak=0)]


((array([1.77561   , 1.77561   , 1.85690619]),
  array([1.67662373, 1.76892016, 1.87113658]),
  -100.07300463312764),
 -100.06554016873194)

In [188]:
1.85690619*180/np.pi #H2O angle

106.39288763871774

In [181]:
mpa_pb_with_energy(oh2optg[:2],["F","H"],AF_3[:2],AH_2[:2,:2,:,:],ep_h20_hf[-1]),hfen

[Bond(0, 1, weak=0)]


((array([1.77561]), array([1.69305788]), -100.07271407813809),
 -100.06554016873194)

In [158]:
hfoptg

array([[0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.69574139]])

## CH$_4$ -> NH$_3$

In [10]:
l=3.3376
x=-(1/24)**.5*l
z=(2/3)**.5*l+x
ch4str="C 0 0 0; H 0 0  {}; H  0  {} {}; H {} {} {}; H {} {} {}".format(z,l/3**.5,x,l/2,-l*3**.5/6,x,-l/2,-l*3**.5/6,x)

In [11]:
CH4=gto.M(atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf = scf.RHF(CH4)
mf.scf()
g0_ch4=mf.Gradients().grad()
h0_ch4=mf.Hessian().hess()
ch4coords=CH4.atom_coords()

converged SCF energy = -40.2151804034983
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000     0.0000000000
1 H     0.0000000000     0.0000000000    -0.0000007926
2 H     0.0000000000    -0.0000007473     0.0000002642
3 H    -0.0000006472     0.0000003736     0.0000002642
4 H     0.0000006472     0.0000003736     0.0000002642
----------------------------------------------


In [159]:
ch4en=-40.2151804034983

In [12]:
af_1ch4=aaff(mf,[1,0,0,0,-1])+alc_deriv_grad_nuc(CH4,[1,0,0,0,-1])

In [13]:
dl=.1
CH4_p=FcM(fcs=[dl,0,0,0,-dl],atom=ch4str,unit="bohr",\
          basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf_p = scf.RHF(CH4_p)
mf_p.scf(dm0=mf_p.init_guess_by_1e())
h0_pch4=mf_p.Hessian().hess()
af_pch4=aaff(mf_p,[1,0,0,0,-1])+alc_deriv_grad_nuc(CH4_p,[1,0,0,0,-1])

converged SCF energy = -41.6026363391302


In [14]:
dl=.1
CH4_m=FcM(fcs=[-dl,0,0,0,dl],atom=ch4str,unit="bohr",\
          basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf_m = scf.RHF(CH4_m)
mf_m.scf(dm0=mf_m.init_guess_by_1e())
h0_mch4=mf_m.Hessian().hess()
af_mch4=aaff(mf_m,[1,0,0,0,-1])+alc_deriv_grad_nuc(CH4_m,[1,0,0,0,-1])

converged SCF energy = -38.8771643174896


In [15]:
af_2ch4=af_1ch4+(af_pch4-af_mch4)/2/dl/2
af_3ch4=af_1ch4+(af_pch4-af_mch4)/2/dl/2+(af_pch4-2*af_1ch4+af_mch4)/6/dl**2
ah_1ch4=h0_ch4+(h0_pch4-h0_mch4)/2/dl
ah_2ch4=h0_ch4+(h0_pch4-h0_mch4)/2/dl+(h0_pch4-2*h0_ch4+h0_mch4)/2/dl**2

# Clear ghost atom gradient
#af_1ch4[0]+=af_1ch4[4]
#af_1ch4[4]-=af_1ch4[4]
#af_2ch4[0]+=af_2ch4[4]
#af_2ch4[4]-=af_2ch4[4]
af_3ch4[0]+=af_3ch4[4]
af_3ch4[4]-=af_3ch4[4]

In [17]:
1.91817704*180/np.pi

109.90344875089689

In [18]:
norm(nh3optg[1]),anglecenter(nh3optg)

(1.8855199999999999, 108.194)

In [19]:
#ap=AP(mf,sites=[0,4])
#ap_p=AP(mf_p,sites=[0,4])
#ap_m=AP(mf_m,sites=[0,4])
#ap.build_all()
#ap_p.build_all()
#ap_m.build_all()
#ep_ch4_nh3=[]
#ep_ch4_nh3.append(ap.APDFT1(np.asarray([1,-1]))+absec("C","N"))
#ep_ch4_nh3.append(ap.APDFT2(np.asarray([1,-1]))+absec("C","N"))
#ep_ch4_nh3.append(ap.APDFT3(np.asarray([1,-1]))+absec("C","N"))
#ep_ch4_nh3.append(ep_ch4_nh3[2]+(ap_p.third_deriv([1,-1])-ap_m.third_deriv([1,-1]))/2/dl/24 )
#ep_ch4_nh3.append(ep_ch4_nh3[3]+(ap_p.third_deriv([1,-1])+ap_m.third_deriv([1,-1])-\
#                                 2*ap.third_deriv([1,-1]))/dl**2/120 )
#

In [22]:
ep_ch4_nh3

[-53.84414407442253,
 -56.31635197834341,
 -56.20440216525058,
 -56.18281786296406,
 -56.21032241738459]

In [50]:
mpa_pb_with_energy(ch4coords[:4],["N","H","H","H"],af_3ch4[:4],ah_2ch4[:4,:4,:,:],ep_ch4_nh3[4],solve_ic=True)

-0.006135258381257643


(array([2.04385424, 2.04385424, 2.04385424, 1.91063324, 1.91063324,
        1.91063324]),
 array([-0.23292565, -0.2329242 , -0.23292189,  0.06801187,  0.06801505,
         0.06801704]),
 array([1.81092859, 1.81093004, 1.81093235, 1.97864511, 1.97864829,
        1.97865028]),
 array([1.8785112 , 1.87851289, 1.87851545, 1.91817704, 1.91817788,
        1.91817798]),
 -56.22820874197485,
 -56.2220734835936)

# NH$_3$-> H$_2$O

In [24]:
nh3lst=[]
for x in range(4):
    nh3lst.append("NHHH"[x])
    nh3lst.extend(nh3optg[x].tolist())
    print(norm(nh3optg[x]))

0.0
1.8855199999999999
1.88552
1.8855199999999999


In [52]:
nh3srt=(("{} "*4+";")*4)[:-1].format(*nh3lst)
NH3=gto.M(atom=nh3srt,unit="bohr",basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])})
mf = scf.RHF(NH3)
mf.scf()

g0=mf.Gradients().grad()
h0=mf.Hessian().hess()
NH3coords=NH3.atom_coords()

converged SCF energy = -56.2221836146943
--------------- RHF gradients ---------------
         x                y                z
0 N    -0.0000000000    -0.0000438944    -0.0000198535
1 H     0.0000000000     0.0000230403    -0.0000251260
2 H    -0.0000153601     0.0000104270     0.0000224897
3 H     0.0000153601     0.0000104270     0.0000224897
----------------------------------------------


In [160]:
nh3en=-56.2221836146943

In [53]:
af_1=aaff(mf,[1,0,0,-1])+alc_deriv_grad_nuc(NH3,[1,0,0,-1])

In [62]:
dl=.1
NH3_p=FcM(fcs=[dl,0,0,-dl],atom=nh3srt,unit="bohr",basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])})
mf_p = scf.RHF(NH3_p)
mf_p.scf(dm0=mf_p.init_guess_by_1e())
h0_p=mf_p.Hessian().hess()
af_p=aaff(mf_p,[1,0,0,-1])+alc_deriv_grad_nuc(NH3_p,[1,0,0,-1])

converged SCF energy = -57.9799514221169


In [63]:
NH3_m=FcM(fcs=[-dl,0,0,dl],atom=nh3srt,unit="bohr",basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])})
mf_m = scf.RHF(NH3_m)
mf_m.scf(dm0=mf_m.init_guess_by_1e())
h0_m=mf_m.Hessian().hess()
af_m=aaff(mf_m,[1,0,0,-1])+alc_deriv_grad_nuc(NH3_m,[1,0,0,-1])

converged SCF energy = -54.5169417377182


In [64]:
af_2=af_1+(af_p-af_m)/2/dl/2
af_3=af_1+(af_p-af_m)/2/dl/2+(af_p-2*af_1+af_m)/6/dl**2
ah_1=h0+(h0_p-h0_m)/2/dl
ah_2=h0+(h0_p-h0_m)/2/dl+(h0_p-2*h0+h0_m)/2/dl**2

# Clear ghost atom gradient
#af_1[0]+=af_1[3]
#af_1[3]-=af_1[3]
#af_2[0]+=af_2[3]
#af_2[3]-=af_2[3]
af_3[0]+=af_3[3]
af_3[3]-=af_3[3]

In [65]:
mpa_pb(NH3coords[:3],["O","H","H"],af_3[:3],ah_2[:3,:3,::],solve_ic=True)

(array([1.88552   , 1.88552   , 1.88833539]),
 array([-0.14172546, -0.141723  , -0.00573316]),
 array([1.74379454, 1.743797  , 1.88260223]),
 array([1.7774034 , 1.7774046 , 1.86818723]))

In [66]:
1.86818723*180/np.pi

107.03924361923602

In [67]:
norm(oh2optg[1]),anglecenter(oh2optg)

(1.77561, 106.393)

In [69]:
ap=AP(mf,sites=[0,3])
ap_p=AP(mf_p,sites=[0,3])
ap_m=AP(mf_m,sites=[0,3])
ap.build_all()
ap_p.build_all()
ap_m.build_all()

In [72]:
ep_nh3_h20=[]
ep_nh3_h20.append(ap.APDFT1(np.asarray([1,-1]))+absec("N","O"))
ep_nh3_h20.append(ap.APDFT2(np.asarray([1,-1]))+absec("N","O"))
ep_nh3_h20.append(ap.APDFT3(np.asarray([1,-1]))+absec("N","O"))
ep_nh3_h20.append(ep_nh3_h20[2]+(ap_p.third_deriv([1,-1])-ap_m.third_deriv([1,-1]))/2/dl/24 )
ep_nh3_h20.append(ep_nh3_h20[3]+(ap_p.third_deriv([1,-1])+ap_m.third_deriv([1,-1])-\
                                 2*ap.third_deriv([1,-1]))/dl**2/120 )

In [73]:
ep_nh3_h20

[-73.53896972665096,
 -76.1653617587322,
 -76.04808577434461,
 -76.03837042976484,
 -76.06316452518669]

In [74]:
mpa_pb_with_energy(NH3coords[:3],["O","H","H"],af_3[:3],ah_2[:3,:3,::],ep_nh3_h20[4],solve_ic=True)

-0.0020020640225670763


(array([1.88552   , 1.88552   , 1.88833539]),
 array([-0.14172546, -0.141723  , -0.00573316]),
 array([1.74379454, 1.743797  , 1.88260223]),
 array([1.7774034 , 1.7774046 , 1.86818723]),
 -76.07005350225101,
 -76.06805143822845)

# H$_2$O -> HF

In [98]:
oh2lst=[]
for x in range(3):
    oh2lst.append("OHH"[x])
    oh2lst.extend(oh2optg[x].tolist())  
oh2srt=(("{} "*4+";")*3)[:-1].format(*oh2lst)

In [99]:
OH2=gto.M(atom=oh2srt,unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])})
mf = scf.RHF(OH2)
mf.scf()
G0=mf.Gradients().grad()
H0=mf.Hessian().hess()
OH2coords=OH2.atom_coords()
AF_1=aaff(mf,[1,0,-1])+alc_deriv_grad_nuc(OH2,[1,0,-1])

converged SCF energy = -76.0633193328123
--------------- RHF gradients ---------------
         x                y                z
0 O    -0.0000132540    -0.0000106032    -0.0000170848
1 H     0.0000054323     0.0000043459     0.0000100623
2 H     0.0000078216     0.0000062573     0.0000070225
----------------------------------------------


In [161]:
h2oen=-76.0633193328123

In [100]:
OH2p=FcM(fcs=[dl,0,-dl],atom=oh2srt,unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])})
mf_p = scf.RHF(OH2p)
mf_p.scf(dm0=mf_p.init_guess_by_1e())
AF_1p=aaff(mf_p,[1,0,-1])+alc_deriv_grad_nuc(OH2p,[1,0,-1])
H0p=mf_p.Hessian().hess()

converged SCF energy = -78.224168134286


In [104]:
OH2m=FcM(fcs=[-dl,0,dl],atom=oh2srt,unit="bohr",basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])})
mf_m = scf.RHF(OH2m)
mf_m.scf(dm0=mf_m.init_guess_by_1e())
AF_1m=aaff(mf_m,[1,0,-1])+alc_deriv_grad_nuc(OH2m,[1,0,-1])
H0m=mf_m.Hessian().hess()

converged SCF energy = -73.958053309612


In [105]:
AF_2=AF_1+(AF_1p-AF_1m)/dl/2/2
AF_3=AF_1+(AF_1p-AF_1m)/2/dl/2+(AF_1p-2*AF_1+AF_1m)/6/dl**2
AH_1=H0+(H0p-H0m)/2/dl
AH_2=H0+(H0p-H0m)/2/dl+(H0p-2*H0+H0m)/2/dl**2
AF_2[0]+=AF_2[2]
AF_2[2]-=AF_2[2]
AF_3[0]+=AF_3[2]
AF_3[2]-=AF_3[2]

In [106]:
mpa_pb(OH2coords[:2],["F","H"],AF_3[:2],AH_2[:2,:2,:,:],solve_ic=True)

(array([1.77561]),
 array([-0.10171737]),
 array([1.67389263]),
 array([1.69305788]))

In [107]:
norm(OH2coords[1]),norm(OH2coords[2])

(1.77561, 1.77561)

In [108]:
fhoptg

array([[0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.69574139]])

In [109]:
ap=AP(mf,sites=[0,2])
ap_p=AP(mf_p,sites=[0,2])
ap_m=AP(mf_m,sites=[0,2])
ap.build_all()
ap_p.build_all()
ap_m.build_all()

In [110]:
ep_h20_hf=[]
ep_h20_hf.append(ap.APDFT1(np.asarray([1,-1]))+absec("O","F"))
ep_h20_hf.append(ap.APDFT2(np.asarray([1,-1]))+absec("O","F"))
ep_h20_hf.append(ap.APDFT3(np.asarray([1,-1]))+absec("O","F"))
ep_h20_hf.append(ep_h20_hf[2]+(ap_p.third_deriv([1,-1])-ap_m.third_deriv([1,-1]))/2/dl/24 )
ep_h20_hf.append(ep_h20_hf[3]+(ap_p.third_deriv([1,-1])+ap_m.third_deriv([1,-1])-\
                                 2*ap.third_deriv([1,-1]))/dl**2/120 )

In [111]:
ep_h20_hf

[-97.39593192349187,
 -100.17510873725631,
 -100.05087063272529,
 -100.04687699517902,
 -100.07046394855858]

In [116]:
mpa_pb_with_energy(OH2coords[:2],["F","H"],AF_3[:2],AH_2[:2,:2,:,:],ep_h20_hf[-1],solve_ic=True)

-0.0005370324164710833


(array([1.77561]),
 array([-0.10171737]),
 array([1.67389263]),
 array([1.69305788]),
 -100.07271407813809)

In [93]:
HF=gto.M(atom="F 0 0 0;H 0 0 1.69574139 ",unit="bohr",basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])})
mf_hf = scf.RHF(HF)
mf_hf.scf()

converged SCF energy = -100.065540168732


-100.06554016873194

In [162]:
hfen=-100.06554016873194

In [118]:
HFT=gto.M(atom="F 0 0 0;H 0 0 1.7756 ",unit="bohr",basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])})
mf_hfT = scf.RHF(HFT)
eThf=mf_hfT.scf()
gThf=mf_hfT.Gradients().grad()
hThf=mf_hfT.Hessian().hess()

converged SCF energy = -100.06345246538
--------------- RHF gradients ---------------
         x                y                z
0 F     0.0000000000    -0.0000000000    -0.0497183403
1 H    -0.0000000000     0.0000000000     0.0497183403
----------------------------------------------


In [119]:
mpa_pb_with_energy(OH2coords[:2],["F","H"],gThf,hThf,eThf,solve_ic=True)

-0.0004602492088642792


(array([1.77561]),
 array([-0.09459553]),
 array([1.68101447]),
 array([1.697896]),
 -100.0654828908125)

In [126]:
np.save("ch4_nh3_data",np.asarray([af_3ch4,ah_2ch4,af_3,ah_2,AF_3,AH_2,ep_h20_hf,ep_nh3_h20,ep_ch4_nh3]))

array([array([[-0.04143357, -0.02392168, -0.01691518],
       [-0.00402351, -0.00232298,  0.06388629],
       [-0.00402351,  0.05945825, -0.02348555],
       [ 0.0494806 , -0.03321359, -0.02348555],
       [ 0.        ,  0.        ,  0.        ]]),
       array([[[[ 2.31884181e-01, -1.12388160e-01, -7.94704297e-02],
         [-1.12388160e-01,  3.61658850e-01, -4.58822740e-02],
         [-7.94704297e-02, -4.58822740e-02,  3.94102517e-01]],

        [[-4.47490142e-02,  2.55038728e-02,  1.46859529e-02],
         [ 2.55038728e-02, -7.41983498e-02,  8.47893885e-03],
         [-3.48931037e-02, -2.01455428e-02, -2.75705150e-01]],

        [[-4.47490141e-02,  2.23473401e-02,  1.91499642e-02],
         [-2.43962427e-02, -2.56981965e-01,  4.44782240e-02],
         [ 3.56763164e-02,  7.31027057e-02, -9.29215343e-02]],

        [[-2.04810928e-01,  6.80155464e-02,  4.80942541e-02],
         [ 1.14759129e-01, -9.69200510e-02, -5.65475654e-03],
         [ 8.11469585e-02, -5.65475653e-03, -9.29215343e

In [127]:
#np.save("ch4_nh3_data",[af_3ch4,ah_2ch4,af_3,ah_2,AF_3,AH_2,ep_h20_hf,ep_nh3_h20,ep_ch4_nh3])

[-53.84414407442253,
 -56.31635197834341,
 -56.20440216525058,
 -56.18281786296406,
 -56.21032241738459]